In [1]:
import numpy as np
import pandas as pd

#load csv file
df = pd.read_csv('/Users/ishratjahanananya/Documents/REPOS/TrademarkVista/uspto_db/data/csv_files/trademarks.csv')

#load only first 10 rows in new dataframe
df1 = df.head(100) 

#save the new dataframe to a new csv file
df1.to_csv('/Users/ishratjahanananya/Documents/REPOS/TrademarkVista/uspto_db/data/csv_files/trademarks_100.csv', index=False, header=False)



In [2]:
df1.head()

,category-code,mark-identification,serial-number,Case-File-Owners,status,xml_filename
0,007,PERFECT HARMONY,78636704,"INNOMOTICS GMBH, SIEMENS AKTIENGESELLSCHAFT, S...",800,apc240120.xml
1,001,THREAD MAGIC,86491524,"DPG USA, Inc., DPG USA, Inc., DPG USA, Inc.",702,apc240120.xml
2,003,SIMPLE SHINE.,86912261,"TORQUAY ETRADING LLC, Love And Pieces LLC, Lov...",710,apc240120.xml
3,032,REFRESHING HYDRATION,88642175,"Super Bakery, Inc., Super Bakery, Inc., Super ...",700,apc240120.xml
4,025,SCHOOLWEAR MADE FOR KIDS TO SHINE,88697251,"Lollytogs, Ltd., Lollytogs, Ltd.",606,apc240120.xml


In [ ]:
# !curl -X POST http://127.0.0.1:5000/api/query \
#      -H "Content-Type: application/json" \
#      -d '{"question": "Find trademarks with NTHLIFE in category 40"}'

curl: (52) Empty reply from server


In [1]:
from trademarkvista.TrademarkQA import TrademarkQA
from trademarkvista.SmolLMWrapper import SmolLMWrapper
import graphene
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session, declarative_base
from sqlalchemy import Column, Integer, String, Text

user_question = "Find trademarks with NTHLIFE in category 9"
# Database Setup
engine = create_engine('postgresql://localhost/trademark_db')
Base = declarative_base()

# SQLAlchemy Model
class TrademarkModel(Base):
    __tablename__ = 'trademarks'
    id = Column(Integer, primary_key=True)
    category_code = Column(String)
    mark_identification = Column(Text)
    serial_number = Column(String, unique=True)
    case_file_owners = Column(Text)
    status = Column(String)
    xml_filename = Column(String)


# GraphQL Schema
class Trademark(graphene.ObjectType):
    id = graphene.Int()
    category_code = graphene.String()
    mark_identification = graphene.String()
    serial_number = graphene.String()
    case_file_owners = graphene.String()
    status = graphene.String()
    xml_filename = graphene.String()

class Query(graphene.ObjectType):
    all_trademarks = graphene.List(Trademark)
    trademark_by_serial = graphene.Field(Trademark, serial_number=graphene.String())
    trademarks_by_category = graphene.List(Trademark, category_code=graphene.String())
    search_marks = graphene.List(Trademark, keyword=graphene.String())

    def resolve_all_trademarks(self, info):
        with Session(engine) as session:
            return session.execute(select(TrademarkModel)).scalars().all()

    def resolve_trademark_by_serial(self, info, serial_number):
        with Session(engine) as session:
            return session.execute(
                select(TrademarkModel).filter_by(serial_number=serial_number)
            ).scalar_one_or_none()

    def resolve_trademarks_by_category(self, info, category_code):
        with Session(engine) as session:
            return session.execute(
                select(TrademarkModel).filter_by(category_code=category_code)
            ).scalars().all()

    def resolve_search_marks(self, info, keyword):
        with Session(engine) as session:
            return session.execute(
                select(TrademarkModel).filter(
                    TrademarkModel.mark_identification.ilike(f'%{keyword}%')
                )
            ).scalars().all()


schema = graphene.Schema(query=Query)
print('schema created')
llm_wrapper = SmolLMWrapper()
print('llm wrapper created')
qa_system = TrademarkQA(llm_wrapper, schema)
print('qa system created')
result = qa_system.process_query(user_question)
print(result)

schema created


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Loaded model and tokenizer
Loaded pipeline
Loaded LLM
llm wrapper created
qa system created
process_query-> User question: Find trademarks with NTHLIFE in category 9
get_graphql_query-> User question: Find trademarks with NTHLIFE in category 9


/Users/ishratjahanananya/Documents/REPOS/TrademarkVista/trademarkvista/SmolLMWrapper.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  self.llm = HuggingFacePipeline(pipeline=self.pipe)
/Users/ishratjahanananya/Documents/REPOS/TrademarkVista/trademarkvista/SmolLMWrapper.py:59: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  query_type = self.llm(type_prompt).strip().lower()
/Users/miniconda3/envs/kaggle/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperatur

Query type: what type of trademark search is this question asking for? reply with only one word:
        'search' for keyword search
        'category' for category search
        'serial' for serial number search
        
        question: find trademarks with nthlife in category 9.

a:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/Users/miniconda3/envs/kaggle/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Search parameter: Extract only the search parameter from this question:
        Question: Find trademarks with NTHLIFE in category 9.
        Answer: NTHLIFE
        Fact: NTHLIFE is a trademark of the National Television and Radio Association.

Answer:
Generated GraphQL query: 
            query {
                searchMarks(keyword: "Extract only the search parameter from this question:
        Question: Find trademarks with NTHLIFE in category 9.
        Answer: NTHLIFE
        Fact: NTHLIFE is a trademark of the National Television and Radio Association.

Answer:") {
                    mark_identification
                    serial_number
                    category_code
                    status
                    case_file_owners
                }
            }
{'error': 'Syntax Error GraphQL (3:92) Unterminated string\n\n2:             query {\n3:                 searchMarks(keyword: "Extract only the search parameter from this question:\n                                    